# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [1]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ritwi\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Read your api key environment variable
load_dotenv()
api_key = os.getenv('NEWS_API_KEY')

In [3]:
print(os.getenv('NEWS_API_KEY'))

97ce2fd52cbc40968ca8c03deb990c70


In [4]:
# Create a newsapi client
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key = api_key)

In [5]:
# Fetch the Bitcoin news articles
btc_news = newsapi.get_everything(
    q='bitcoin', 
    language='en',
)

# Fetch the Ethereum news articles
eth_news = newsapi.get_everything(
    q='ethereum', 
    language='en',
)

In [6]:
def create_sentiment_df(news_data):
    sentiments = []

    for article in news_data["articles"]:
        try:
            text = article["content"]
            date = article["publishedAt"][:10]
            sentiment = analyzer.polarity_scores(text)
            compound = sentiment["compound"]
            pos = sentiment["pos"]
            neu = sentiment["neu"]
            neg = sentiment["neg"]
            
            sentiments.append({
                "text": text,
                "date": date,
                "compound": compound,
                "positive": pos,
                "negative": neg,
                "neutral": neu
                
            })
            
        except AttributeError:
            pass
        
    # Create DataFrame
    df = pd.DataFrame(sentiments)

    # Reorder DataFrame columns
    cols = ["date", "text", "compound", "positive", "negative", "neutral"]
    df = df[cols]

    return df

In [7]:
# Create the Bitcoin sentiment scores DataFrame
btc_df = create_sentiment_df(btc_news)
btc_df.head()

,date,text,compound,positive,negative,neutral
0,2022-08-03,It was 8:45 in the morning of June 13 when Bil...,0.5574,0.119,0.000,0.881
1,2022-08-02,"Tools to trace cryptocurrencies have, over jus...",0.0000,0.000,0.000,1.000
2,2022-07-12,Miners voluntarily powered down as energy dema...,0.1531,0.057,0.041,0.902
3,2022-07-25,You gotta hand it to some people for being imp...,0.5994,0.130,0.034,0.835
4,2022-07-16,People in Russia will soon no longer be allowe...,0.0516,0.088,0.057,0.855


In [8]:
# Create the Ethereum sentiment scores DataFrame
eth_df = create_sentiment_df(eth_news)
eth_df.head()

,date,text,compound,positive,negative,neutral
0,2022-07-23,"Less than two weeks into its soft launch, Game...",-0.1531,0.000,0.045,0.955
1,2022-08-02,"It's a day of the week ending in the letter ""y...",-0.2732,0.059,0.115,0.827
2,2022-07-06,Axie Infinity was the prime example of crypto ...,0.2263,0.077,0.000,0.923
3,2022-07-25,"The Ethereum Merge is significant, and not jus...",0.2023,0.048,0.000,0.952
4,2022-07-11,Ethereum and Bitcoin have nearly become househ...,0.2500,0.065,0.000,0.935


In [9]:
# Describe the Bitcoin Sentiment
btc_df.describe()

,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.082528,0.067360,0.046650,0.885970
std,0.432667,0.064107,0.065877,0.081768
min,-0.862500,0.000000,0.000000,0.641000
25%,-0.225725,0.000000,0.000000,0.837250
50%,0.077150,0.061500,0.000000,0.892500
75%,0.383550,0.118250,0.087000,0.941000
max,0.888500,0.271000,0.249000,1.000000


In [10]:
# Describe the Ethereum Sentiment
eth_df.describe()

,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.00000
mean,0.157545,0.084310,0.049170,0.86651
std,0.443830,0.074315,0.067736,0.09236
min,-0.936600,0.000000,0.000000,0.63300
25%,-0.083575,0.000000,0.000000,0.81400
50%,0.208350,0.070500,0.000000,0.88650
75%,0.510600,0.129750,0.086250,0.93325
max,0.921600,0.299000,0.354000,1.00000


### Questions:

Q: Which coin had the highest mean positive score?

A: 

Q: Which coin had the highest compound score?

A: 

Q. Which coin had the highest positive score?

A: 

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [11]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [12]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
sw = set(stopwords.words('english'))

# Expand the default stopwords list if necessary
sw = sw.union({
    'char', 
    'chars',
    'cryptocurrencies',
    'cryptocurrency',
    'crypto',
    'ha',
})


In [13]:
# Complete the tokenizer function
def tokenize(text):
    """Tokenizes text."""

    if type(text) == list:
        text = ' '.join(text)
    
    # Remove the punctuation from text
    regex = re.compile('[^a-zA-Z ]')
    re_clean = regex.sub('', text)

    # Create a tokenized list of the words
    words = word_tokenize(re_clean)
    
    # Lemmatize words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]

    # Convert the words to lowercase and remove the stop words
    output = [word.lower() for word in lem if word.lower() not in sw]
    
    return output

In [ ]:
pip install nltk.download('popular')

In [14]:
# Create a new tokens column for Bitcoin
btc_df['tokens'] = [tokenize(text) for text in btc_df['text']]
btc_df.head()

,date,text,compound,positive,negative,neutral,tokens
0,2022-08-03,It was 8:45 in the morning of June 13 when Bil...,0.5574,0.119,0.000,0.881,"[wa, morning, june, bill, stewart, ceo, maineb..."
1,2022-08-02,"Tools to trace cryptocurrencies have, over jus...",0.0000,0.000,0.000,1.000,"[tools, trace, last, several, year, allowed, l..."
2,2022-07-12,Miners voluntarily powered down as energy dema...,0.1531,0.057,0.041,0.902,"[miners, voluntarily, powered, energy, demand,..."
3,2022-07-25,You gotta hand it to some people for being imp...,0.5994,0.130,0.034,0.835,"[got, ta, hand, people, impetuous, even, face,..."
4,2022-07-16,People in Russia will soon no longer be allowe...,0.0516,0.088,0.057,0.855,"[people, russia, soon, longer, allowed, use, d..."


In [15]:
# Create a new tokens column for Ethereum
eth_df['tokens'] = [tokenize(text) for text in eth_df['text']]
eth_df.head()

,date,text,compound,positive,negative,neutral,tokens
0,2022-07-23,"Less than two weeks into its soft launch, Game...",-0.1531,0.000,0.045,0.955,"[less, two, week, soft, launch, gamestops, nft..."
1,2022-08-02,"It's a day of the week ending in the letter ""y...",-0.2732,0.059,0.115,0.827,"[day, week, ending, letter, inevitably, mean, ..."
2,2022-07-06,Axie Infinity was the prime example of crypto ...,0.2263,0.077,0.000,0.923,"[axie, infinity, wa, prime, example, gaming, l..."
3,2022-07-25,"The Ethereum Merge is significant, and not jus...",0.2023,0.048,0.000,0.952,"[ethereum, merge, significant, people, work, o..."
4,2022-07-11,Ethereum and Bitcoin have nearly become househ...,0.2500,0.065,0.000,0.935,"[ethereum, bitcoin, nearly, become, household,..."


---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [ ]:
from collections import Counter
from nltk import ngrams

In [ ]:
def get_words(corpus):
    corpus_string = ' '.join(corpus)
    processed = tokenize(corpus_string)
    words = Counter(processed)
    output = [word.lower() for word in words if word.lower() not in sw]
    return output

In [ ]:
def get_bigrams(corpus):
    corpus_string = ' '.join(corpus)
    processed = tokenize(corpus_string)
    bigrams = ngrams(processed, n=2)
    bigrams_list = Counter(bigrams)
    return bigrams_list

In [ ]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, ngrams=False, N=10):
    """Returns the top N tokens from the frequency count"""
    top_10 = dict(Counter(tokens).most_common(N))
    columns = ['word', 'count'] if ngrams == False else ['bigram', 'count']
    return pd.DataFrame(list(top_10.items()), columns=columns)

In [ ]:
# Generate the Bitcoin N-grams where N=2
btc_corpus = [article['content'] for article in btc_news['articles']]
btc_bigrams = get_bigrams(btc_corpus)
token_count(btc_bigrams)

In [ ]:
# Generate the Ethereum N-grams where N=2
eth_corpus = [article['content'] for article in eth_news['articles']]
eth_bigrams = get_bigrams(eth_corpus)
token_count(eth_bigrams, ngrams=True)

In [ ]:
# Use token_count to get the top 10 words for Bitcoin
btc_words = tokenize(btc_corpus)
token_count(btc_words)

In [ ]:
# Use token_count to get the top 10 words for Ethereum
eth_words = tokenize(eth_corpus)
token_count(eth_words)

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud
btc_tokens = tokenize(btc_corpus)
btc_wc = WordCloud(colormap='plasma', max_words=50).generate(' '.join(btc_tokens))
plt.title('Bitcoin News Word Cloud', fontdict={'fontsize': 25, 'fontweight': 'bold'})
plt.axis('off')
plt.imshow(btc_wc)
plt.show()

In [ ]:
# Generate the Ethereum word cloud
eth_tokens = tokenize(eth_corpus)
eth_wc = WordCloud(colormap='plasma', max_words=50).generate(' '.join(eth_tokens))
plt.title('Ethereum News Word Cloud', fontdict={'fontsize': 25, 'fontweight': 'bold'})
plt.axis('off')
plt.imshow(eth_wc)
plt.show()

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [ ]:
# Concatenate all of the Bitcoin text together
btc_corpus_str = ' '.join(btc_corpus).replace('\n', '')
btc_corpus_str[:500]

In [ ]:
# Run the NER processor on all of the text
btc_doc = nlp(btc_corpus_str)


# Add a title to the document
btc_doc.user_data["title"] = "Bitcoin NER"

In [ ]:
# Render the visualization
displacy.render(btc_doc, style='ent')

In [ ]:
# List all Entities
print([ent.text for ent in btc_doc.ents])

---

### Ethereum NER

In [ ]:
# Concatenate all of the Ethereum text together
eth_corpus_str = ' '.join(eth_corpus).replace('\n', '')
eth_corpus_str[:500]

In [ ]:
# Run the NER processor on all of the text
eth_doc = nlp(eth_corpus_str)

# Add a title to the document
eth_doc.user_data["title"] = "Ethereum NER"

In [ ]:
# Render the visualization
displacy.render(eth_doc, style='ent')

In [ ]:
# List all Entities
print([ent.text for ent in eth_doc.ents])

---